In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Load the datasets (replace with your actual file paths)
train_data = pd.read_csv(r'C:\Users\Harsh244635\OneDrive - EXLService.com (I) Pvt. Ltd\Documents\Capstone Project\dataset\X_train_cleaned.csv')
test_data = pd.read_csv(r'C:\Users\Harsh244635\OneDrive - EXLService.com (I) Pvt. Ltd\Documents\Capstone Project\dataset\X_train_cleaned.csv')

In [40]:
# Extract features and targets
X_train_cleaned = train_data['review_body'].tolist()  # Text data
Y_train = train_data['sentiment'].tolist()  # Labels
X_test_cleaned = test_data['review_body'].tolist()
Y_test = test_data['sentiment'].tolist()

In [41]:
# Ensure the data is a list of strings
X_train_cleaned = [str(item) for item in X_train_cleaned]
X_test_cleaned = [str(item) for item in X_test_cleaned]

In [42]:
# Split training data into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_cleaned, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

In [43]:
# Convert lists to pandas Series
X_train = pd.Series(X_train)
X_val = pd.Series(X_val)
X_test = pd.Series(X_test_cleaned)

In [44]:
# Define tokenizer and lemmatizer
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# Define function to lemmatize the review text
def lemmatize_text(text):
    st = ""
    for word in w_tokenizer.tokenize(text):
        st += lemmatizer.lemmatize(word) + " "
    return st.strip()  # Strip to remove any trailing spaces

# Apply lemmatization to the 'review' text in the datasets
X_train = X_train.apply(lemmatize_text)
X_val = X_val.apply(lemmatize_text)
X_test = X_test.apply(lemmatize_text)

# Optionally, print a sample to check
print(X_train.head())

0    tried it doe not work i saw the blog of previo...
1    i give this product mixed review in principle ...
2    great product for the price i had some frayes ...
3    first of all i got them for dollar and they su...
4    i can t say much it never functioned and i wa ...
dtype: object

In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [46]:
# Tokenizing text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [47]:
# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [48]:
# Padding sequences to ensure uniform input size
max_length = 100  # You can change this based on your data
X_train_pad = pad_sequences(X_train_seq, padding='post', maxlen=max_length)
X_val_pad = pad_sequences(X_val_seq, padding='post', maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=max_length)

In [49]:
# Convert labels to categorical (one-hot encoding) for multi-class classification
y_train = to_categorical(Y_train, num_classes=3)
y_val = to_categorical(Y_val, num_classes=3)
y_test = to_categorical(Y_test, num_classes=3)

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [34]:
# Define the model
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_length))

# BiLSTM layer
model.add(Bidirectional(LSTM(64, return_sequences=False)))

# Dense layer
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.7))

# Output layer with softmax activation for multi-class classification
model.add(Dense(3, activation='softmax'))  # 3 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',  # Suitable for multi-class classification
              metrics=['accuracy'])

c:\Users\Harsh244635\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(

In [35]:
# Training the model
history = model.fit(
    X_train_pad, y_train,
    epochs=3,  # Number of epochs
    batch_size=32,  # You can adjust based on your data
    validation_data=(X_val_pad, y_val),
    verbose=2
)

# Evaluating the model on test data
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 1/3
8582/8582 - 387s - 45ms/step - accuracy: 0.4101 - loss: 1.0637 - val_accuracy: 0.5346 - val_loss: 0.9671
Epoch 2/3
8582/8582 - 412s - 48ms/step - accuracy: 0.4921 - loss: 0.9572 - val_accuracy: 0.6064 - val_loss: 0.8662
Epoch 3/3
8582/8582 - 425s - 49ms/step - accuracy: 0.5514 - loss: 0.8934 - val_accuracy: 0.6340 - val_loss: 0.8147
10727/10727 - 163s - 15ms/step - accuracy: 0.6363 - loss: 0.8104
Test accuracy: 0.6363

In [36]:
# Save the model
model.save('biLSTM_model_2.h5')

In [37]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,142,795 (15.80 MB)

 Trainable params: 1,380,931 (5.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,761,864 (10.54 MB)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
# Load the pre-trained BiLSTM model
model = tf.keras.models.load_model(r'C:\Users\Harsh244635\OneDrive - EXLService.com (I) Pvt. Ltd\Documents\Capstone Project\biLSTM_model_2.h5')

# Define the maximum length of sequences (same as what was used during training)
max_sequence_length = 100  # Adjust based on the training config

# Recreate the Tokenizer used during training (assuming you don't have the original tokenizer)
vocab_size = 10000  # You can adjust this if you know the vocab size used during training

tokenizer = Tokenizer(num_words=vocab_size)

In [27]:
# Function to preprocess the input sentence
def preprocess_text(sentences):
    # Tokenize and pad the input sentences
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
    
    # Debugging: Print out the number of sentences after preprocessing
    print(f"Number of processed (padded) sentences: {len(padded_sequences)}")
    
    return padded_sequences

In [28]:
# Function to predict sentiment for a list of sentences
def predict_sentiments(sentences):
    processed_sentences = preprocess_text(sentences)
    
    # Debugging: Print out the shape of the processed sentences
    print(f"Shape of processed sentences: {processed_sentences.shape}")
    
    # Predicting sentiment for the input sentences
    predictions = model.predict(processed_sentences)
    
    # Debugging: Print out the shape of predictions
    print(f"Shape of predictions: {predictions.shape}")
    
    # The prediction is a vector of probabilities for each class: [positive, negative, neutral]
    sentiment_classes = np.argmax(predictions, axis=1)  # Get the index of the max probability for each sentence
    
    # Map the index to sentiment labels
    sentiment_labels = ['Positive', 'Negative', 'Neutral']
    predicted_labels = [sentiment_labels[i] for i in sentiment_classes]
    
    return predicted_labels

In [30]:
# Example usage:
sentence = "I love this product"
sentiment = predict_sentiments(sentence)
print(f"The sentiment of the sentence is: {sentiment}")

Number of processed (padded) sentences: 19
Shape of processed sentences: (19, 100)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Shape of predictions: (19, 3)
The sentiment of the sentence is: ['Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral']

In [21]:
from sklearn.metrics import classification_report

In [22]:
# Example test dataset (sentences and true labels)
test_sentences = [
    "I love this product!",
    "This is the worst thing I have ever bought.",
    "It's okay, neither good nor bad."
]

In [31]:
# True labels for the test dataset (replace these with actual labels from your dataset)
true_labels = ['Positive', 'Negative', 'Neutral']

In [32]:
# Check the number of sentences
print(f"Number of sentences in test data: {len(test_sentences)}")
print(f"True labels: {true_labels}")

Number of sentences in test data: 3
True labels: ['Positive', 'Negative', 'Neutral']

In [33]:


# Generate predictions for the test dataset
predictions = predict_sentiments(test_sentences)

Number of processed (padded) sentences: 3
Shape of processed sentences: (3, 100)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Shape of predictions: (3, 3)

In [34]:
# Ensure predictions and true labels are in a list format (if not already)
predictions = np.array(predictions)  # Convert predictions to a NumPy array
true_labels = np.array(true_labels)  # Convert true labels to a NumPy array

In [37]:
# Print each sentence with its predicted sentiment
print("Predictions for given sentences:")
for sentence, prediction in zip(test_sentences, predictions):
    print(f"Sentence: '{sentence}' => Predicted Sentiment: {prediction}")

Predictions for given sentences:
Sentence: 'I love this product!' => Predicted Sentiment: Negative
Sentence: 'This is the worst thing I have ever bought.' => Predicted Sentiment: Neutral
Sentence: 'It's okay, neither good nor bad.' => Predicted Sentiment: Neutral

In [35]:
# Check if the number of predictions matches the number of true labels
print(f"Number of predictions: {len(predictions)}")
print(f"Number of true labels: {len(true_labels)}")

Number of predictions: 3
Number of true labels: 3

In [36]:
# Generate the classification report
print("Classification Report:\)
print(classification_report(true_labels, predictions, target_names=['Positive', 'Negative', 'Neutral']))

Classification Report:

              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00         1
    Negative       0.50      1.00      0.67         1
     Neutral       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.17      0.33      0.22         3
weighted avg       0.17      0.33      0.22         3


c:\Users\Harsh244635\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Harsh244635\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Harsh244635\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [51]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# Load the saved BiLSTM model
model = load_model('biLSTM_model_2.h5')

# Load the test dataset (assuming it's in CSV format)
test_data = pd.read_csv(r'C:\Users\Harsh244635\OneDrive - EXLService.com (I) Pvt. Ltd\Documents\Capstone Project\dataset\X_train_cleaned.csv')

# Extract test features and labels
X_test_cleaned = test_data['review_body'].tolist()
Y_test = test_data['sentiment'].tolist()

# Ensure the data is a list of strings
X_test_cleaned = [str(item) for item in X_test_cleaned]

# Apply lemmatization to the test set (same function as during training)
X_test = pd.Series(X_test_cleaned).apply(lemmatize_text)

# Tokenize the test data using the tokenizer trained on the training set
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length as the training data
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=100)  # Use the same maxlen as during training

# Make predictions using the trained model
y_pred = model.predict(X_test_pad)

# Convert probabilities to class labels (using np.argmax to get the class with the highest probability)
y_pred_classes = np.argmax(y_pred, axis=1)

# Encode the true labels (if they are in string format, like "positive", "negative", "neutral")
# Convert to numeric labels if the labels are not already numeric
label_encoder = LabelEncoder()
y_test_classes = label_encoder.fit_transform(Y_test)



10727/10727 ━━━━━━━━━━━━━━━━━━━━ 347s 32ms/step

In [53]:
y_test_classes

array([2, 2, 2, ..., 1, 2, 2], dtype=int64)

In [54]:
y_pred_classes

array([2, 2, 2, ..., 1, 2, 2], dtype=int64)

In [55]:
# Convert y_test_classes and y_pred_classes to lists
y_test_classes_list = y_test_classes.tolist()
y_pred_classes_list = y_pred_classes.tolist()

In [59]:
# Generate the classification report
report = classification_report(y_test_classes_list, y_pred_classes_list)
print(report)

              precision    recall  f1-score   support

           0       0.58      0.82      0.68    114418
           1       0.53      0.30      0.39    114419
           2       0.78      0.78      0.78    114419

    accuracy                           0.64    343256
   macro avg       0.63      0.64      0.62    343256
weighted avg       0.63      0.64      0.62    343256
